In [1]:
import numpy as np 
import pandas as pd 

# 將資料做切割(太長的會被分開/或404的)，帶入第一個model預測name並且儲存

In [15]:
# train_full_content.csv 這張表的是由主辦單位提供後經過爬蟲得來的
df = pd.read_csv('train_full_content.csv' ,encoding='utf-8',index_col=0)

In [16]:
df.head()
# hyperlink:連結
#全文
#aml人物名單

,hyperlink,article,name_list
0,https://news.cnyes.com/news/id/4352432,理財基金量化交易追求絕對報酬 有效對抗牛熊市鉅亨網記者 鄭心芸2019/07/05 22:3...,[]
1,https://udn.com/news/story/120775/4112519,10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件，北市府勞動局認定業者未依職業...,[]
2,https://www.mirrormedia.mg/story/20191008soc011/,社會2019.10.08 09:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何從...,[]
3,http://domestic.judicial.gov.tw/abbs/wkw/WHD9H...,例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進家...,[]
4,http://www.coolloud.org.tw/node/93356,內政部都市計畫委員會委員為審查大社工業區是否降為乙種工業區，將於8月30日到高雄大社現勘（註...,[]


In [18]:
name_list = df.name_list

In [19]:
all_name_list = []
for i in range(len(name_list)):
    ii = [j for j in name_list.iloc[i][1:-1].replace(" ", "").split("'") if len(j)>=2]
    s = []
    for k in ii:
        #print(k)
        s.append(k)
    all_name_list.append(s) 

In [20]:
df['name_list'] = all_name_list

In [21]:
def split_content(x):
    
    if len(x)<=500:
        return [x]
    elif (len(x)>=500) and(len(x)<1000):
        return [ x[:500+3],x[500-6:] ]
    
    elif (len(x)>=1000) and (len(x)<1500):
        return [ x[:500+3],x[500-3:1000+3] ,x[1000-3:]]
    
    elif (len(x)>=1500) and (len(x)<2000):
        return [ x[:500+3],x[500-3:1000+3] ,x[1000-3:1500+3], x[1500-3:]]
    
    else:
        return [x[:500+3],x[500-3:1000+3] ,x[1000-3:1500+3], x[1500-3:2000-3] , x[2000-3:2000-3+500] ]
    


In [22]:
# 將長篇文章依據字數500左右去分割_list裡面有三至五個不等的文章
df['article_split'] = df['article'].apply(lambda x :split_content(x))

In [23]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences #2.2.4
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [24]:
from tqdm import tqdm, trange

In [25]:
tokenizer_chinese = BertTokenizer.from_pretrained("bert-base-chinese", do_lower_case=False)

In [26]:
# 定義model的class
tag_values = ['O',
 'B_person_name',
 'M_person_name',
 'E_person_name',
 'PAD']
tag2idx = { 'O': 0,
           'B_person_name': 1,
           'M_person_name': 2,
           'E_person_name': 3,
           'PAD': 4}

In [29]:
# load model
PATH = 'step1_1output_bertmode_step1_ner.pth'#'bertmode_asia.pth'
model = torch.load(PATH)
model.eval()

C:\Users\User\Anaconda3\envs\torch_2\lib\site-packages\torch\serialization.py:657: SourceChangeWarning: source code of class 'transformers.modeling_bert.BertForTokenClassification' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
C:\Users\User\Anaconda3\envs\torch_2\lib\site-packages\torch\serialization.py:657: SourceChangeWarning: source code of class 'transformers.modeling_bert.BertModel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
C:\Users\User\Anaconda3\envs\torch_2\lib\site-packages\torch\serialization.py:657: SourceChangeWarning: source code of class 'transformers.modeling_bert.BertEmbeddings' has changed. you can ret

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [30]:
#　找出每篇文章所有的名字
cols_name = [ ]
for p in range(len(df)):
    
    row_name = []
    for sentence in df['article_split'].iloc[p]:
        # bert預測
        tokenized_sentence = tokenizer_chinese.encode(sentence)
        input_ids = torch.tensor([tokenized_sentence]).cuda()

        with torch.no_grad():
            output = model(input_ids)
        label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
        tokens = tokenizer_chinese.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
        new_tokens, new_labels = [], []
        for token, label_idx in zip(tokens, label_indices[0]):
            if token.startswith("##"):
                new_tokens[-1] = new_tokens[-1] + token[2:]
            else:
                new_labels.append(tag_values[label_idx])#ex:['O','O','O','O',...]
                new_tokens.append(token)# ex:['[CLS]', '益', '公', '司', '債', '或', '新',...]
        texto=''
        for i in range(len(new_labels)):
            if new_labels[i] !='O':
                texto+= new_tokens[i]
            else:
                texto+='O' #'OOO張堯勇OOOOOOOOOOOOO'
        for i in texto.split('O'):
            if len(i)>1:#['張堯勇', '張堯勇']，單一個字或者空白的會被削去
                 row_name.append(i)
    uniq_name = list(set(row_name)) #['鄭心芸', '巴菲特', '詹姆斯·西蒙斯', '堯勇', '索羅斯', '張堯勇']
    cols_name.append(uniq_name)

In [31]:
list(set(row_name))

['李佳泓', '黃維本', '鄭心芸']

In [36]:
cols_name[:10]

[['張堯勇', '巴菲特', '詹姆斯·西蒙斯', '鄭心芸'],
 ['賴香伶', '康水順'],
 ['張慶輝', '李育材', '德唐', '李日順', '凱德唐', '君鴻'],
 ['沈君融', '陳世恒'],
 ['張勝富', '王敏玲', '陳其邁', '韓國瑜', '邱花妹', '江丙坤', '吳忠穎', '余陳月英', '黃捷', '江炳坤'],
 ['強森', '川普', '任正非', '孟晚舟'],
 ['雪克來提．扎克爾', '扎克爾', '艾爾肯·吐尼亞孜', '徐海榮', '吳妍'],
 ['周永暉', '祁文中'],
 [],
 ['王瑞宏', '黃懷德']]

In [37]:
df['all_name'] = cols_name

In [38]:
df.head()

,hyperlink,article,name_list,article_split,all_name
0,https://news.cnyes.com/news/id/4352432,理財基金量化交易追求絕對報酬 有效對抗牛熊市鉅亨網記者 鄭心芸2019/07/05 22:3...,[],[理財基金量化交易追求絕對報酬 有效對抗牛熊市鉅亨網記者 鄭心芸2019/07/05 22:...,"[張堯勇, 巴菲特, 詹姆斯·西蒙斯, 鄭心芸]"
1,https://udn.com/news/story/120775/4112519,10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件，北市府勞動局認定業者未依職業...,[],[10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件，北市府勞動局認定業者未依職...,"[賴香伶, 康水順]"
2,https://www.mirrormedia.mg/story/20191008soc011/,社會2019.10.08 09:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何從...,[],[社會2019.10.08 09:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何...,"[張慶輝, 李育材, 德唐, 李日順, 凱德唐, 君鴻]"
3,http://domestic.judicial.gov.tw/abbs/wkw/WHD9H...,例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進家...,[],[例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進...,"[沈君融, 陳世恒]"
4,http://www.coolloud.org.tw/node/93356,內政部都市計畫委員會委員為審查大社工業區是否降為乙種工業區，將於8月30日到高雄大社現勘（註...,[],[內政部都市計畫委員會委員為審查大社工業區是否降為乙種工業區，將於8月30日到高雄大社現勘（...,"[張勝富, 王敏玲, 陳其邁, 韓國瑜, 邱花妹, 江丙坤, 吳忠穎, 余陳月英, 黃捷, ..."


In [39]:
import pickle
df.to_pickle("step2_1_output_train_full_data.pkl")